In [17]:
%%pyspark
df = spark.read.load('abfss://raspdata@datalakesu20220516.dfs.core.windows.net/sensor=1984/year=2022/month=05/data2022_05_16_10_59_33.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(, , , SessionStarting, )

SynapseWidget(Synapse.DataFrame, cba01a88-6693-4bd7-947a-d4fc4528ef67)

In [ ]:
df.createOrReplaceTempView('mytempTable')

In [ ]:
%%sql
SELECT *
FROM mytempTable

In [ ]:
spark.sql('')

In [5]:
df = spark.read.load('abfss://matador@asadatalake2prbthc.dfs.core.windows.net/*.json', format='json',multiline=True)
display(df.limit(10))

StatementMeta(mysparkpool, 2, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5a8f3415-8838-4cd1-a526-ce666488be27)

In [6]:
from pyspark.sql.functions import explode
childrenDF=df.select('lastName',explode('children').alias('childrenflat'))
childrenDF.show()

StatementMeta(mysparkpool, 2, 7, Finished, Available)

In [7]:
newDF=childrenDF.select('lastName', 'childrenflat.firstName','childrenflat.gender','childrenflat.grade')
newDF.show()


StatementMeta(mysparkpool, 2, 8, Finished, Available)

In [8]:
%%sql
CREATE DATABASE synapsesparkDB

StatementMeta(mysparkpool, 2, 9, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
newDF.write.format("parquet").saveAsTable('synapsesparkdb.matadorChildren')

StatementMeta(mysparkpool, 2, 11, Finished, Available)

In [11]:
%%sql
SELECT * FROM synapsesparkdb.matadorChildren

StatementMeta(mysparkpool, 2, 13, Finished, Available)

<Spark SQL result set with 5 rows and 4 fields>

In [13]:
%%spark
// VI ER I SCALA - den 
// Vi indlæser via Scala
val dfScala = spark.sql("SELECT * FROM synapsesparkdb.matadorchildren")
dfScala.show()


StatementMeta(mysparkpool, 2, 16, Finished, Available)

In [14]:
%%spark
// Write the dataframe into your sql pool
import org.apache.spark.sql.SqlAnalyticsConnector._
import com.microsoft.spark.sqlanalytics.utils.Constants

val sql_pool_name = "SQLPool01" 

dfScala.write.sqlanalytics(s"$sql_pool_name.dbo.MatadorChildren", Constants.INTERNAL)


StatementMeta(mysparkpool, 2, 18, Finished, Available)

In [15]:
%%spark
// Read  the table we just created in the sql pool as a Spark dataframe
val spark_read = spark.read.
    sqlanalytics(s"$sql_pool_name.dbo.MatadorChildren")
spark_read.show(5, truncate = false)

StatementMeta(mysparkpool, 2, 19, Finished, Available)

In [ ]:
%%spark
// Write the dataframe into your sql pool
import org.apache.spark.sql.SqlAnalyticsConnector._
import com.microsoft.spark.sqlanalytics.utils.Constants

dfScala.write.
    option(Constants.DATA_SOURCE, "synapsedlcontainer").
    option(Constants.FILE_FORMAT, "parquetfileformat").
    sqlanalytics("superusersDW.dbo.MatadorChildrenExt", Constants.EXTERNAL)

In [ ]:
%%sql


